## Transfer learning - Application à la classification d'images

Dans ce TP, nous allons utiliser la technique du transfer learning pour réaliser un classificateur d'images capable de distinguer différents types de nuages :
  * cirrus
  * cumulus
  * cumulonimbus

Ce TP est basé sur la leçon n°1 du cours en ligne http://www.fast.ai/.

__Attention !__ La version de la librairie fastai utilisée est la 0.7. Ce TP n'est pas compatible avec fastai v1.0 et supérieures.

In [ ]:
# A mettre en haut de chaque notebook afin d'activer certaines fonctionnalités de jupyter notebook

# Autoreload
%reload_ext autoreload
%autoreload 2

# Permet d'afficher les graphiques dans le notebook
%matplotlib inline

## Import des librairies

Nous allons utiliser la librairie de haut niveau fastai.

Cette librairie permet de réaliser en très peu de lignes de code certaines tâches classiques de Deep Learning, telles que la classification d'image.

In [ ]:
# This file contains all the main external libs we'll use
from fastai.imports import *

In [ ]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

`PATH` est le chemin d'accès à vos données.

`sz` est la taille à laquelle vos images seront redimensionnées, afin d'obtenir un entraînement rapide.

In [ ]:
PATH = "data/nuages/"
sz=224

Vérifie si le GPU est disponible sur votre machine.

In [ ]:
torch.cuda.is_available()

## Jetons un oeil à nos images

Pour utiliser les fonctions de classification d'image de la librairie fastai, il faut classer les images dans des répertoires *train* et *valid*. Dans chacun de ces répertoires, il faut un sous-répertoire par classe (dans notre exemple, *cirrus*, *cumulus* et *cumulonimbus*).

In [ ]:
os.listdir(PATH)

In [ ]:
os.listdir(f'{PATH}valid')

In [ ]:
# A quoi ressemblent nos noms de fichiers ?
files = os.listdir(f'{PATH}valid/cirrus')[:5]
files

In [ ]:
# A quoi ressemble une image de cirrus ?
img = plt.imread(f'{PATH}valid/cirrus/{files[0]}')
plt.imshow(img);

Voici à quoi ressemble la donnée brute pour une image :

In [ ]:
img.shape

In [ ]:
img[:4,:4]

## C'est parti pour notre premier modèle !

Nous allons utiliser un modèle <b>pré-entraîné</b>, c'est-à-dire un modèle créé par quelqu'un d'autre pour classifier d'autres types d'images. Ici nous allons utiliser un modèle entraîné sur le jeu de données ImageNet (1,2 millions d'images et 1000 classes). Ce modèle est un réseau de neurones convolutionnel.

Nous allons utiliser le modèle <b>resnet34</b>. Il s'agit du réseau de neurones qui a remporté le challenge ImageNet en 2015.

Plus d'infos sur ce modèle : [resnet models](https://github.com/KaimingHe/deep-residual-networks)

Et hop, voici comment entraîner notre classificateur de nuages en quatre lignes de code !

In [ ]:
# Décommenter cette ligne si vous souhaitez recalculer les activations
# (en cas de modification du jeu de données images)
#shutil.rmtree(f'{PATH}tmp', ignore_errors=True)

In [ ]:
arch=resnet34
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz))
learn = ConvLearner.pretrained(arch, data, precompute=True)
learn.fit(0.01, 20)

Que vaut notre modèle ? Il a une précision d'environ 80%, ce qui n'est pas si mal vu le peu d'efforts consentis... Nous verrons plus tard comment faire mieux.

## Analyse des résultats : regardons quelques images

En plus des métriques telles que le coût et la précision, il est utile de regarder qualitativement le fonctionnement du modèle :
1. Quelques images correctement classifiées, prises au hasard
2. Quelques images mal classifiées, prises au hasard
3. Les images les plus correctes de chaque classe (celles pour lesquelles le modèle attribue une probabilité élevée d'appartenance à la classe)
4. Les images les plus incorrectes de chaque classe (celles pour lesquelles le modèle attribue une probabilité élevée d'appartenance à une autre classe)
5. Les images les plus incertaines

In [ ]:
# Labels du jeu de validation
data.val_y

In [ ]:
# Permet de savoir à quelle classe correspond le label 0, 1 et 2
data.classes

In [ ]:
# Calcul des prédictions pour le jeu de validation. Predictions en échelle logarithmique
log_preds = learn.predict()
log_preds.shape

In [ ]:
log_preds[:10]

In [ ]:
preds = np.argmax(log_preds, axis=1)  # convertit les log probabilities en classe 0, 1 et 2
probs = np.exp(log_preds)        # probabilités en échelle [0,1]

In [ ]:
def rand_by_mask(mask): return np.random.choice(np.where(mask)[0], 4, replace=False)
def rand_by_correct(is_correct): return rand_by_mask((preds == data.val_y)==is_correct)

In [ ]:
def plot_val_with_title(idxs, title):
    imgs = np.stack([data.val_ds[x][0] for x in idxs])
    title_probs = [probs[x] for x in idxs]
    print(title)
    return plots(data.val_ds.denorm(imgs), rows=1, titles=title_probs)

In [ ]:
def plots(ims, figsize=(12,6), rows=1, titles=None):
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        sp.axis('Off')
        if titles is not None: sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i])

In [ ]:
def load_img_id(ds, idx): return np.array(PIL.Image.open(PATH+ds.fnames[idx]))

def plot_val_with_title(idxs, title):
    imgs = [load_img_id(data.val_ds,x) for x in idxs]
    title_probs = [probs[x] for x in idxs]
    print(title)
    return plots(imgs, rows=1, titles=title_probs, figsize=(16,8))

In [ ]:
# 1. A few correct labels at random
plot_val_with_title(rand_by_correct(True), "Correctly classified")

In [ ]:
# 2. A few incorrect labels at random
plot_val_with_title(rand_by_correct(False), "Incorrectly classified")

In [ ]:
def most_by_mask(mask, y):
    idxs = np.where(mask)[0]
    return idxs[np.argsort(-probs[idxs, y])[:4]]

def most_by_correct(y, is_correct): 
    return most_by_mask(((preds == data.val_y)==is_correct) & (data.val_y == y), y)

In [ ]:
plot_val_with_title(most_by_correct(0, True), "Cirrus les plus corrects")

In [ ]:
plot_val_with_title(most_by_correct(1, True), "Cumulonimbus les plus corrects")

In [ ]:
plot_val_with_title(most_by_correct(2, True), "Cumulus les plus corrects")

In [ ]:
plot_val_with_title(most_by_correct(0, False), "Cirrus les plus incorrects")

In [ ]:
plot_val_with_title(most_by_correct(1, False), "Cumulonimbus les plus incorrects")

In [ ]:
plot_val_with_title(most_by_correct(2, False), "Cumulus les plus incorrects")

In [ ]:
most_uncertain = np.argsort(np.abs(probs[:,0] - 0.5))[:4]
plot_val_with_title(most_uncertain, "Probabilité de cirrus les plus proches de 0.5 (incertain)")

In [ ]:
most_uncertain = np.argsort(np.abs(probs[:,1] - 0.5))[:4]
plot_val_with_title(most_uncertain, "Probabilité de cumulonimbus les plus proches de 0.5 (incertain)")

In [ ]:
most_uncertain = np.argsort(np.abs(probs[:,2] - 0.5))[:4]
plot_val_with_title(most_uncertain, "Probabilité de cumulus les plus proches de 0.5 (incertain)")

## Améliorons notre modèle

### Data augmentation

Si vous continuez d'entraîner votre modèle en augmentant le nombre d'épochs, vous vous apercevrez que le modèle va *overfitter*. D'une certaine manière, il va apprendre par coeur les images du jeu d'entraînement, mais deviendra moins performant losqu'il s'agira de généraliser sur le jeu de validation.

Une solution pour éviter ce phénomène est d'ajouter des données d'entraînement. Pour cela, deux solutions :

  * Aller chercher d'autres images. C'est évidemment une solution performante, mais potentiellement longue et coûteuse.
  * Faire de la *data augmentation*. 

Nous allons mettre en oeuvre cette seconde solution. La data augmentation va consister à modifier les images lors de l'entraînement, en leur appliquant différentes transformations : miroir, zoom, rotation...

Pour cela, nous allons utiliser la fonctionnalité de data augmentation de la librairie fastai. La librairie dispose de fonctions de data augmentation prédéfinies. Nous allons utiliser :

  * `transforms_side_on` : rotations et symétrie gauche/droite (pas de symétrie haut/bas, afin de garder les objets "la tête en haut")
  * `max_zoom` : zoome dans l'image

In [ ]:
tfms = tfms_from_model(resnet34, sz, aug_tfms=transforms_side_on, max_zoom=1.1)

In [ ]:
def get_augs():
    data = ImageClassifierData.from_paths(PATH, bs=2, tfms=tfms, num_workers=1)
    x,_ = next(iter(data.aug_dl))
    return data.trn_ds.denorm(x)[1]

In [ ]:
ims = np.stack([get_augs() for i in range(6)])

In [ ]:
plots(ims, rows=2)

Let's create a new `data` object that includes this augmentation in the transforms.

In [ ]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms)
learn = ConvLearner.pretrained(arch, data, precompute=False)

In [ ]:
learn.fit(1e-2, 20)

Maintenant que nous avons un bon modèle, nous pouvons le sauvegarder.

In [ ]:
learn.save('224_lastlayer')

In [ ]:
learn.load('224_lastlayer')

### Fine-tuning

Maintenant que nous avons entraîné la dernière couche, nous pouvons essayer de faire un fine-tuning des autres couches. Pour dire à la librairie que nous voulons *dégeler* les poids de l'ensemble des couches, nous allons utiliser `unfreeze()`.

In [ ]:
learn.unfreeze()

Notez que les autres couches ont déjà été entraînées pour reconnaître les photos imagenet (alors que nos couches finales étaient initialisées aléatoirement). Il faut donc veiller à ne pas détruire les poids qui ont déjà été soigneusement entraînés.

D'une manière générale, les couches précédentes ont des caractéristiques plus générales. Par conséquent, nous nous attendons à ce qu'ils aient besoin de moins de réglages pour les nouveaux jeux de données. Pour cette raison, nous utiliserons différents learning rates pour différentes couches: les premières couches seront à 1e-4, les couches intermédiaires à 1e-3, et pour nos couches FC nous partirons à 1e-2 comme auparavant. FastAI appelle cela le *differential learning rate*, bien qu'il n'y ait pas de nom officiel dans la littérature.

In [ ]:
lr=np.array([1e-4,1e-3,1e-2])

In [ ]:
learn.fit(lr, 6)

Est-ce que le modèle est meilleur ?

A votre avis, que s'est-il passé ?

In [ ]:
learn.save('224_all')

In [ ]:
learn.load('224_all')

Une dernière chose que nous pouvons faire avec l'augmentation des données est de l'utiliser lors de l'inférence. C'est une technique appellée *Test Time Augmentation* (TTA).

TTA effectue simplement des prédictions non seulement sur les images du jeu de validation, mais également avec un certain nombre de versions augmentées de manière aléatoire (par défaut, l'image originale et 4 versions augmentées de façon aléatoire). Il prend alors la prédiction moyenne de ces images.

C'est le même principe qu'en prévision d'ensemble météo.

In [ ]:
log_preds,y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)

In [ ]:
accuracy_np(probs, y)

On peut en général espérer autour de 10 à 20% de réduction de l'erreur grâce au TTA.

## Analyse des résultats

### Confusion matrix 

In [ ]:
preds = np.argmax(probs, axis=1)
#probs = probs[:,1]

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, preds)

Nous pouvons afficher la matrice de contingence sous une forme graphique (ce qui est pratique lorsqu'on a un nombre important de classes).

In [ ]:
plot_confusion_matrix(cm, data.classes)

### Revoyons nos images

In [ ]:
plot_val_with_title(most_by_correct(0, False), "Cirrus les plus incorrects")

In [ ]:
plot_val_with_title(most_by_correct(1, False), "Cumulonimbus les plus incorrects")

In [ ]:
plot_val_with_title(most_by_correct(2, False), "Cumulus les plus incorrects")

## Résumé :
<b>Comment entraîner un classificateur d'image au top niveau international grâce au transfer learning !</b>

1. Récupérer un modèle pré-entraîné
1. Entraîner la dernière couche pour quelques epochs, en veillant à ne pas overfitter
1. Entraîner la dernière couche avec data augmentation
1. Dégeler les autres couches et entraîner pour le fine-tuning